# Agentic Page RAG

### 1. Imports

In [12]:
from typing_extensions import TypedDict, Annotated
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import display, Markdown
from dotenv import load_dotenv

import os

from scripts import utils

load_dotenv()

True

### 2. Configurations

In [3]:
LLM_MODEL="qwen3:latest"
BASE_URL="http://localhost:11434"

llm = ChatOllama(
    model=LLM_MODEL,
    base_url=BASE_URL
)


In [6]:
utils.extract_filters("what is amazon's annual cashflow in 2024?")

{'company_name': 'amazon', 'doc_type': '10-k', 'fiscal_year': 2024}

### 3. Defining Agent State

In [8]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

### 4. Retrieval Tool

In [15]:
@tool
def retriever_docs(query: str, k: int = 5):
    """
    Retrieve relevant financial documents from ChromaDB.
    Extracts filters and retrieves matching documents.

    Args:
        query: Search the query (e.g "what was amazon's revenue in q3 2025?")
        k: Number of documents to retrieve. Generaly prefer 5 docs
    
    Returns:
        Retrieved documents with with metadata as formatted string
    """
    filters = utils.extract_filters(query)
    ranking_keywords = utils.generate_ranking_keywords(query)
    results = utils.search_docs(query, filters, ranking_keywords, k=k)

    docs = utils.rank_documents_by_keywords(results, ranking_keywords, k=k)
    
    print(f"[RETRIEVED] {len(docs)} documents")

In [16]:
query = "What is amazon's revenue in 2024?"

retriever_docs.invoke(query)

  [1] Doc 1: Score=13.0027
  [2] Doc 0: Score=8.2653
  [3] Doc 2: Score=6.2568
[RETRIEVED] 3 documents
